<a href="https://colab.research.google.com/github/MazurovaNN/A_B_test/blob/main/DZ8_A_B_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from typing import Union
from tqdm import tqdm

import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

##Задание
На сайте запущен А/В тест с целью увеличить доход. В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue.
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

In [11]:
df = pd.read_excel('gb_sem_8_hm.xlsx')

/usr/local/lib/python3.9/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [12]:
df.head()

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0


##Подготовка данных

In [13]:
df.shape

(10000, 3)

In [14]:
df.USER_ID.nunique()

6324

In [15]:
df = df.groupby(['USER_ID', 'VARIANT_NAME'], as_index=False).agg({'REVENUE': 'sum'})

In [16]:
df.shape

(7865, 3)

In [17]:
df.groupby('USER_ID', as_index=False).agg({'VARIANT_NAME': 'count'})['VARIANT_NAME'].value_counts()

1    4783
2    1541
Name: VARIANT_NAME, dtype: int64

##Посмотрим на группы отдельно

In [18]:
unique_ids = \
(df
 .groupby('USER_ID', as_index=False)
 .agg({'VARIANT_NAME': 'count'})
 #.['VARIANT_NAME'].value_counts()
 .query('VARIANT_NAME == 1')
 .USER_ID
 .values
 )

In [19]:
df_new = df[df.USER_ID.isin(unique_ids)].copy(deep=True)

In [22]:
df_new.describe()

,USER_ID,REVENUE
count,4783.000000,4783.000000
mean,4994.395777,0.135873
std,2898.618472,3.011392
min,2.000000,0.000000
25%,2476.000000,0.000000
50%,4975.000000,0.000000
75%,7515.000000,0.000000
max,9998.000000,196.010000


In [23]:
unique_ids = \
(df
 .groupby('USER_ID', as_index=False)
 .agg({'VARIANT_NAME': 'count'})
 #.['VARIANT_NAME'].value_counts()
 .query('VARIANT_NAME == 2')
 .USER_ID
 .values
 )

In [24]:
df_new = df[df.USER_ID.isin(unique_ids)].copy(deep=True)

In [25]:
df_new.describe()

,USER_ID,REVENUE
count,3082.000000,3082.000000
mean,4952.189487,0.111807
std,2878.641386,1.833932
min,3.000000,0.000000
25%,2444.000000,0.000000
50%,4962.000000,0.000000
75%,7476.000000,0.000000
max,10000.000000,58.630000


Пока мы еще ничего не посчитали, но уже можно заметить, что максимальный доход принесенный пользователем revenueв первой группе сильно (почти в 4 раза) больше, чем в группе B. Все мы знаем, что среднее очень неустойчиво к выбросам, так что нам необходимо будет это учесть.

##Применение статистических критериев

In [40]:
def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [42]:
def proportion_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Вероятность событий
    size = control.loc[:, column].shape[0]
    prop_control = control.loc[:, column].sum() / size
    prop_treatment = treatment.loc[:, column].sum() / size
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = stats.bernoulli.rvs(p=prop_control, size=size)
        treatment_sample = stats.bernoulli.rvs(p=prop_treatment, size=size)
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size = proportion.proportion_effectsize(prop_control, prop_treatment)
    
    power = zt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

##Метрика доход принесенный пользователем revenue.

In [47]:
fig = px.histogram(data[data['revenue'] == 1],
                   x='revenue',
                   color = 'ab_group',
                   title='revenue_distribution',
                   marginal = 'box',
                   nbins = 50,
                   barmode='overlay')
fig.show()

In [50]:
continious_result(control[control.purchase == 1], 
                  treatment[treatment.purchase == 1],
                  column='revenue')

100%|██████████| 10000/10000 [00:03<00:00, 2636.54it/s]


,effect_size,alpha,beta,CI,difference
revenue,-0.0999,0.255976,0.796248,"[-2.038, 19.376]",6.323717


In [51]:
fig = px.histogram(data, x="revenue",
                   color='ab_group', barmode='group',
                   height=400)
fig.show()

In [52]:
proportion_result(control, treatment, column='revenue')

100%|██████████| 10000/10000 [00:09<00:00, 1067.89it/s]


,effect_size,alpha,beta,CI,difference
revenue,0.344271,0.0,0.0,"[0.148, 0.172]",0.16001


In [53]:
import plotly.express as px
fig = px.histogram(data,
                   x='revenue',
                   color = 'ab_group',
                   title='revenue_distribution',
                   marginal = 'box',
                   nbins = 100,
                   barmode='overlay')

fig.show()

In [54]:
continious_result(control, treatment, column='revenue')

100%|██████████| 10000/10000 [00:11<00:00, 902.27it/s]


,effect_size,alpha,beta,CI,difference
revenue,-0.016711,0.202166,0.749424,"[-0.034, 0.448]",0.158646


По результатам эксперимента, проанализировав полученные данные, можно дать следующие рекомендации:
1) полученные alpha и beta больше допустимого порога
2) доверительный интервал включает ноль
Т.о. полученный результат не статистически значимый
Следовательно не можем отвергнуть нулевую гипотезу в пользу альтернативной.
Мощности теста вполне достаточно хватило для объема выборки.
Рекомендации: тест следует перезапустить